In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import KFold

In [2]:
### load training index
indexfile = '/home/kai/data/kaggle/talkingdata/wl/data/trainset/train_index.npy'
train_index = np.load(indexfile)

### load feature cols
import json as js
featurefile = '/home/kai/data/kaggle/talkingdata/wl/data/features/feature_cols.json'
with open(featurefile, 'r') as myjs:
    added_features = js.load(myjs)

In [4]:
path = '/home/kai/data/kaggle/talkingdata/data/'
train = pd.read_csv(path + 'train_cleaned_final.csv')
test = pd.read_csv(path + 'test_cleaned_final.csv')

In [5]:
history_index = list(set(range(len(train))) - set(train_index))

In [6]:
import sys
print(sys.getsizeof(train)/ 1024 **3)

15.154054783284664


# get history and train

In [7]:
df_history = train.iloc[history_index].copy()
df_train = train.iloc[train_index].copy()


In [8]:
import sys
print('history size: {}, length is: {}'.format(sys.getsizeof(df_history)/ 1024 **3, len(df_history))   )
print('train size: {}, length is: {}'.format(sys.getsizeof(df_train)/ 1024 **3, len(df_train))   )


history size: 12.0613476857543, length is: 134903890
train size: 4.470348380506039, length is: 50000000


In [9]:
df_train.columns

Index(['ip', 'app', 'device', 'os', 'channel', 'day', 'hour', 'timestamp',
       'minute', 'second', 'is_attributed'],
      dtype='object')

In [10]:
orders = {}
feature_col = ['ip', 
              'app', 
              'device', 
              'os', 
              'channel',
              'day',
              'hour',
              'minute',
              'second']

# feature_col = ['ip', 
#               'app', 
#               'device', 
#               'os', 
#               'channel']
for col in feature_col:
    orders[col] = 10 ** (int(np.log(max(train[col].max(),test[col].max() ) + 1) / np.log(10)) + 1)
def get_group(df, cols):
    """
    define an encoding method which can ganrantee the adding value will be unique.
    eg: artist_name_composer will be a combination of (artist_name,composer) and the encoding will reflect the unqiue combination of those two
    """
    group = df[cols[0]].copy()
    for col in cols[1:]:
        group = group * orders[col] + df[col]
        
    return group

import gc
# del train
gc.collect()

60

In [11]:
orders

{'app': 1000,
 'channel': 1000,
 'day': 100,
 'device': 10000,
 'hour': 100,
 'ip': 1000000,
 'minute': 100,
 'os': 1000,
 'second': 100}

# count
plan 1. count from historical data  
plan 2. count from all data

In [12]:
def count(df_history, df_train, cols, target=None):
    """
    Purpose: add a new feature to training df.count the number of records for each feature combination (eg, artist_name_composer)) 
    """
    
    group = get_group(df_train, cols)
    group_all = get_group(df_history, cols)
    
    count_map = group_all.value_counts()
    
    return group.map(count_map).fillna(0)

# mean
mean P(target | feature combination)

purpose: add a new feature to training df. conditional probability P(replay (target) | feature combination (eg, artist_name_composer))
Get the conditional Probability only from historical data and apply to train data.

P(replay | X feature combination) = P( replay & X feature combination) / P (X feature combination)  
=(count(replay & X feature combination) / count(total)) / (count(X feature combination) / count(total))  
= count(replay & X feature combination) / count(X feature combination)  
= sum((replay & X feature combination)) / count(X feature combination)  
= sum((replay or not replayed & X feature combination)) / count(X feature combination)# since replay is 1, not replay is 0  
= sum( X feature combination) / count(X feature combination)  
= mean(X feature combination)  

In [13]:
def scaller(num):
    sca = 1
    while num * sca < 1:
        sca *= 10
    return sca

def mean(df_history, df_train, cols, target):
    """
    Purpose: add a new feature to training df. conditional probability P(replay (target) | feature combination (eg, artist_name_composer)) 
    """
  
    # encoding df's cols into a new series
    group = get_group(df_train, cols)
    # encoding df_history's cols into a new series
    group_history = get_group(df_history, cols)
    # get the conditional probability p(target| feature combination. eg, artist_name_composer) 
    mean_map = df_history.groupby(group_history)[target].mean()
    # mean_map: key - encoding, value - target mean
#     ### sca
#     m_min = mean_map[mean_map > 0].min()
#     sca = scaller(m_min)
#     mean_map *= sca
#     ###

    return group.map(mean_map).fillna(-1)

# reversemean
reverse mean P(feature combination | target)

In [14]:
def reversemean(df_history, df_train, cols, target):
    """
    Purpose: add a new feature to training df. conditional probability P(replay (target) | feature combination (eg, artist_name_composer)) 
    """
  
    # encoding df's cols into a new series
    group = get_group(df_train, cols)
    # encoding df_history's cols into a new series
    group_history = get_group(df_history, cols)
    # get the conditional probability p(target| feature combination. eg, artist_name_composer) 
    positive = group_history[df_history[target] == 1]
    negative = group_history[df_history[target] == 0]
    index_p = set(positive.unique())
    index_n = set(negative.unique())
    index_n.difference_update(index_p)
    map_reverse_p = positive.groupby(positive).count() / len(positive)
    map_reverse_n = pd.Series(np.zeros(len(index_n)), index=index_n)
    map_reverse = pd.concat([map_reverse_p, map_reverse_n])
    return group.map(map_reverse).fillna(-1)

# generate all cols

In [15]:
from itertools import combinations


combine_col = ['ip', 
              'app', 
              'device', 
              'os', 
              'channel', 
              'day',
              'hour',
              'minute',
              'second']

def col_name(cols, func=None):
    if func is None:
        return '_'.join(cols)
    else:
        return '_'.join(cols) + '_' + func.__name__

counter = 0



for num_col in [1,2,3]:
    for cols in combinations(combine_col, num_col):
        for func in [count, mean]:
            feature_name = col_name(cols, func=func)
            if feature_name in added_features:
                counter += 1
#                 df_train[feature_name] = func(df_history, df_train, cols, target='is_attributed')
#                 test[feature_name] = func(df_history, test, cols, target='is_attributed')
                df_train[feature_name] = func(train, df_train, cols, target='is_attributed')
                test[feature_name] = func(train, test, cols, target='is_attributed')
                gc.collect()
                train_str = 'train {}:   {}   \t\t\t size: {} G.'.format(counter, feature_name, sys.getsizeof(df_train)/ 1024 **3)
                test_str = 'test {}:   {}   \t\t\t size: {} G.'.format(counter, feature_name, sys.getsizeof(test)/ 1024 **3)
                print(train_str)
                print(test_str)
                print('------')
                with open('feature_all.txt', 'w') as text_file:
                    text_file.write(train_str + '\n')
                    text_file.write(test_str + '\n')
                    text_file.write('------' + '\n')

train 1:   ip_mean   			 size: 4.84287741035223 G.
test 1:   ip_mean   			 size: 1.5399990379810333 G.
------
train 2:   ip_app_mean   			 size: 5.2154064401984215 G.
test 2:   ip_app_mean   			 size: 1.679998941719532 G.
------
train 3:   ip_channel_count   			 size: 5.587935470044613 G.
test 3:   ip_channel_count   			 size: 1.8199988454580307 G.
------
train 4:   ip_channel_mean   			 size: 5.960464499890804 G.
test 4:   ip_channel_mean   			 size: 1.9599987491965294 G.
------
train 5:   ip_minute_count   			 size: 6.332993529736996 G.
test 5:   ip_minute_count   			 size: 2.099998652935028 G.
------
train 6:   ip_second_mean   			 size: 6.705522559583187 G.
test 6:   ip_second_mean   			 size: 2.2399985566735268 G.
------
train 7:   device_minute_mean   			 size: 7.0780515894293785 G.
test 7:   device_minute_mean   			 size: 2.3799984604120255 G.
------
train 8:   ip_app_device_mean   			 size: 7.45058061927557 G.
test 8:   ip_app_device_mean   			 size: 2.519998364150524 G.
------

# Saving Files

In [17]:
target = 'is_attributed'
train_col = []
train_col = added_features.copy()
train_col.append(target)

df_train = df_train[train_col]
test = test[added_features]
df_train.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/features/train_all_5000k_17cols.csv', index=False)
print('training saving done!')
test.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/features/test_all_5000k_17cols.csv', index=False)
print('testing saving done!')

training saving done!
testing saving done!


In [ ]:
train_col